In [4]:
import pandas as pd
import numpy as np
import matplotlib.pylot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'matplotlib.pylot'

In [ ]:
data_1 = "C:\Users\danie\Downloads\Benavente.csv"
data_2 = "C:\Users\danie\Downloads\Cartaxo.csv"
data_3 = "C:\Users\danie\Downloads\Almeirim.csv"

df_Benavente 
df_